In [ ]:
import rasterio
from pyproj import Transformer
import pandas as pd


In [ ]:
ndvi = rasterio.open("data/ndvi_map.tif")

ndvi_to_wgs84 = Transformer.from_crs(ndvi.crs, "EPSG:4326", always_xy=True)
ndvi_from_dwgs84 = Transformer.from_crs("EPSG:4326", ndvi.crs, always_xy=True)

ndvi_band = ndvi.read(1)
y = pd.DataFrame(ndvi_band)
y.describe()